In [1]:
import os.path
!cd ~/bigdata/coursework/
filepath = os.path.abspath('.')
print("absolute path: ",filepath)

absolute path:  /home/enterprise.internal.city.ac.uk/acvn470/bigdata/coursework


In [11]:
#Load files from HDFS
!wget http://saltdean.nsqdc.city.ac.uk:50070/webhdfs/v1/data/movielens/ml-20m/README.txt?op=OPEN -O README.txt
!wget http://saltdean.nsqdc.city.ac.uk:50070/webhdfs/v1/data/movielens/ml-20m/links.csv?op=OPEN -O links.csv
!wget http://saltdean.nsqdc.city.ac.uk:50070/webhdfs/v1/data/movielens/ml-20m/movies.csv?op=OPEN -O movies.csv
!wget http://saltdean.nsqdc.city.ac.uk:50070/webhdfs/v1/data/movielens/ml-20m/ratings.csv?op=OPEN -O ratings.csv
!wget http://saltdean.nsqdc.city.ac.uk:50070/webhdfs/v1/data/movielens/ml-20m/tags.csv?op=OPEN -O tags.csv
! ls -l

--2018-04-01 12:19:31--  http://saltdean.nsqdc.city.ac.uk:50070/webhdfs/v1/data/movielens/ml-20m/README.txt?op=OPEN
Resolving saltdean.nsqdc.city.ac.uk (saltdean.nsqdc.city.ac.uk)... 10.207.1.106
Connecting to saltdean.nsqdc.city.ac.uk (saltdean.nsqdc.city.ac.uk)|10.207.1.106|:50070... connected.
HTTP request sent, awaiting response... 307 TEMPORARY_REDIRECT
Location: http://seaford.nsqdc.city.ac.uk:50075/webhdfs/v1/data/movielens/ml-20m/README.txt?op=OPEN&namenoderpcaddress=saltdean.nsqdc.city.ac.uk:8020&offset=0 [following]
--2018-04-01 12:19:31--  http://seaford.nsqdc.city.ac.uk:50075/webhdfs/v1/data/movielens/ml-20m/README.txt?op=OPEN&namenoderpcaddress=saltdean.nsqdc.city.ac.uk:8020&offset=0
Resolving seaford.nsqdc.city.ac.uk (seaford.nsqdc.city.ac.uk)... 10.207.1.85
Connecting to seaford.nsqdc.city.ac.uk (seaford.nsqdc.city.ac.uk)|10.207.1.85|:50075... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10261 (10K) [application/octet-stream]
Saving to: ‘README.txt’


In [2]:
!pwd


/home/enterprise.internal.city.ac.uk/acvn470/bigdata/coursework


In [1]:
from pyspark.ml import *
#from pyspark.ml.classification import *
from pyspark.ml.feature import *
from pyspark.ml.param import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *
from pyspark.sql import *
import pixiedust
from pyspark.sql.types import Row
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()


Pixiedust database opened successfully


In [2]:
from pyspark.sql.types import *

def load_data(fraction, train_fraction, seed):
    schema = StructType([
    StructField("userId", IntegerType()),
    StructField("movieId",IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", IntegerType())
    ])
    df_ratings = spark.read.csv("ratings.csv", header=True, mode="DROPMALFORMED", schema=schema)
    schema = StructType([
    StructField("movieId", IntegerType()),
    StructField("title",StringType()),
    StructField("genres", StringType())
    ])
    df_movies = spark.read.csv("movies.csv", header=True, mode="DROPMALFORMED", schema=schema)
    schema = StructType([
    StructField("userId", IntegerType()),
    StructField("movieId",IntegerType()),
    StructField("tag", StringType()),
    StructField("timestamp", IntegerType())
    ])
    df_tags = spark.read.csv("tags.csv", header=True, mode="DROPMALFORMED", schema=schema)
    df_ratings = df_ratings.sample(False, fraction, seed)
    df_ratings_train, df_ratings_test = df_ratings.randomSplit([train_fraction, 1-train_fraction])
    return df_ratings_train, df_ratings_test ,df_movies, df_tags

df_ratings_train, df_ratings_test, df_movies, df_tags = load_data(0.1, 0.8, 42)

# Displays the content of the DataFrame to stdout
df_ratings_train.show()
df_movies.show()
df_tags.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    151|   4.0|1094785734|
|     1|    293|   4.0|1112484703|
|     1|    919|   3.5|1094785621|
|     1|   1215|   4.0|1094786082|
|     1|   1249|   4.0|1112485382|
|     1|   1278|   4.0|1094785986|
|     1|   1374|   4.0|1094785746|
|     1|   2644|   3.5|1112484994|
|     1|   3081|   4.0|1094786143|
|     1|   3153|   4.0|1112485816|
|     1|   4105|   3.5|1094786104|
|     1|   6502|   3.5|1094786048|
|     1|   6755|   3.5|1094786067|
|     1|   7164|   3.5|1112485923|
|     1|   7438|   4.0|1112484789|
|     1|   8482|   3.5|1112485781|
|     2|    469|   3.0| 974820598|
|     2|   3173|   4.0| 974820869|
|     2|   3565|   3.0| 974820776|
|     2|   3703|   4.0| 974821014|
+------+-------+------+----------+
only showing top 20 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+

In [9]:
df_ratings_train.printSchema()
df_movies.printSchema()
df_tags.printSchema()
# Create a (temporary) view using the DataFrame, so that we can us use SparkSQL.
df_ratings_train.createOrReplaceTempView("ratings")
df_movies.createOrReplaceTempView("movies")
df_tags.createOrReplaceTempView("tags")


# SQL can now be run on the DataFrame. 
# Let's start by selecting only the ratings elements of each row 
results = spark.sql("SELECT DISTINCT rating FROM ratings")
results.show()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)

+------+
|rating|
+------+
|   3.5|
|   4.5|
|   2.5|
|   1.0|
|   4.0|
|   0.5|
|   3.0|
|   2.0|
|   1.5|
|   5.0|
+------+



In [4]:
from pyspark.ml.recommendation import *
from pyspark.ml import Pipeline, PipelineModel, Transformer, Estimator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator, TrainValidationSplitModel
from pyspark.ml.evaluation import Evaluator, RegressionEvaluator
from pyspark.mllib.evaluation import RankingMetrics, RegressionMetrics
from pyspark.sql.functions import col, expr
from pyspark import keyword_only

#custom pre-processing transformer: 
class SelectSize(Transformer):
    def _transform(self, df_data, fraction=1, seed=42):
        df_data = df_data.sample(False, fraction, seed)
        return df_data

#custom post processing transformer: removes nan entries in predicted ratings
class NanRemover(Transformer):
    def _transform(self, df_pred):
        df_nonnull = df_pred.dropna(subset=['prediction', ])
        df_pred = df_nonnull.withColumn('prediction', df_nonnull['prediction'].cast('double'))
        return df_pred

def rank_mean_precision_old(df_ratings, df_pred, threshold):
    df_topuser_ratings = df_ratings.where("rating > "+'{:0.1f}'.format(threshold)).groupBy("userId")\
      .agg(expr("collect_set(movieId) as movies"))
    df_topuser_pred = df_pred.orderBy(col("userId"), expr("prediction DESC")).groupBy("userId")\
      .agg(expr("collect_list(movieId) as movies"))
    df_topuser_ratingsvspred = df_topuser_ratings.join(df_topuser_pred, ["userId"]).rdd\
      .map(lambda row: (row[1], row[2][:15]))
    ranks = RankingMetrics(df_topuser_ratingsvspred)
    return ranks.meanAveragePrecision

def rank_mean_precision(df_pred, threshold):
    df_topuser_ratings = df_pred.where("rating > "+'{:0.1f}'.format(threshold)).groupBy("userId")\
      .agg(expr("collect_set(movieId) as movies"))
    df_topuser_pred = df_pred.orderBy(col("userId"), expr("prediction DESC")).groupBy("userId")\
      .agg(expr("collect_list(movieId) as movies"))
    df_topuser_ratingsvspred = df_topuser_ratings.join(df_topuser_pred, ["userId"]).rdd\
      .map(lambda row: (row[1], row[2][:15]))  ############# CHECKKKKKKKKKKKKKKKKK<<<<<<<=======================
    ranks = RankingMetrics(df_topuser_ratingsvspred)
    return ranks.meanAveragePrecision

#custom evaluator: using rank meanAveragePrecision
class RankEvaluator(Evaluator):
    @keyword_only
    def __init__(self, threshold=None):
        super(RankEvaluator, self).__init__()
        self.threshold = Param(self, 'threshold', 'Top K')  ####topK??
        self._setDefault(threshold=2.5)
        kwargs = self.__init__._input_kwargs
        self.setParams(**kwargs)
    @keyword_only
    def setParams(self, threshold=None):
        kwargs = self.setParams._input_kwargs
        return self._set(**kwargs)
    def isLargerBetter(self):
        return True
    def setThreshold(self, value):
        self._paramMap[self.threshold] = value
        return self
    def getThreshold(self):
        return self.getOrDefault(self.threshold)
    def _evaluate(self, df_pred):
        threshold = self.getThreshold()
        return rank_mean_precision(df_pred, threshold)
    


In [5]:
#Define pipeline steps
#selectsize = SelectSize(fraction=0.5)
als = ALS(rank=10, maxIter=5, seed=0, regParam=0.01,
          userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True)
nanremover = NanRemover()
#non-negative matrix factorisation se https://www.cs.uic.edu/~liub/KDD-cup-2007/proceedings/matrix-fact-Wu.pdf
#end define pipeline steps

#fit model
pipeline = Pipeline(stages=[als, nanremover])
model = pipeline.fit(df_ratings_train)

#predict and evaluate prediction results
df_train_pred = model.transform(df_ratings_train)
df_test_pred = model.transform(df_ratings_test)


evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")
%time
print ("RMSE - training:",evaluator.evaluate(df_train_pred))
%time
print ("RMSE  - testing:",evaluator.evaluate(df_test_pred))

evaluator = RankEvaluator(threshold=2.5)
%time
print ("RankMetric - training:",evaluator.evaluate(df_train_pred))
%time
print ("RankMetric  - testing:",evaluator.evaluate(df_test_pred))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs
RMSE - training: 0.6106812859309652
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs
RMSE  - testing: 1.0882116360062546
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
RankMetric - training: 0.7826697707366967
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.15 µs
RankMetric  - testing: 0.8908760494658923


In [10]:
paramgrid = ParamGridBuilder()\
    .addGrid(als.rank,[5,10,20,40])\
    .build()
    
tvs = TrainValidationSplit(estimator=pipeline, estimatorParamMaps=paramgrid, evaluator=evaluator, trainRatio=0.75)
%time
tvsmodel = tvs.fit(df_ratings_train)
best_model = TrainValidationSplitModel(tvs).bestModel

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs


In [19]:
%time
df_train_pred = tvsmodel.transform(df_ratings_train)
%time
df_test_pred = tvsmodel.transform(df_ratings_test)
%time
print ("Best RankMetric - training:",evaluator.evaluate(df_train_pred))
%time
print ("Best RankMetric  - testing:",evaluator.evaluate(df_test_pred))


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.2 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.82 µs
Best RankMetric - training: 0.7790226172639618
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


Exception ignored in: <bound method JavaModelWrapper.__del__ of <pyspark.mllib.evaluation.RankingMetrics object at 0x7f72587b5080>>
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/mllib/common.py", line 142, in __del__
    self._sc._gateway.detach(self._java_model)
AttributeError: 'RankingMetrics' object has no attribute '_sc'


Best RankMetric  - testing: 0.8911543489777336


In [15]:
df_train_pred

0.7790316648009347

In [16]:
df_test_pred

0.8911425259017919